In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Input

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

model_name = 'GAN_GEN'
model_train_epochs_step=100
model_train_epochs_stop=1000

try:
    os.mkdir(model_name)
except:
    print("Error creating directory!")

trainData = pd.read_csv("train.csv")
# Do we need to shuffle our train set???
trainData = trainData.sample(frac=1)
trainX = trainData.drop(columns=["Class"])
trainY = trainData.Class

testData = pd.read_csv("test.csv")
testX = testData.drop(columns=["Class"])
testY = testData.Class

combined = pd.DataFrame({"iterations":[], "auroc":[], "auprc": []})

In [ ]:
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=30,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=60))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=90))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(30, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer="adam")
    return generator

def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(90,input_dim=30))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(60))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(30))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer="adam")
    return discriminator

def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan


In [ ]:
generator = create_generator()
discriminator = create_discriminator()
gan = create_gan(discriminator, generator)


In [ ]:
for e in range(1,10000):

    noise= np.random.normal(0,1, [500, 100])
    synthetic_fakes = generator.predict(noise)
    
    selection = np.random.randint(low=0,high=trainData.shape[0],size=500)
    trainBatchX = trainX.values[selection]
    trainBatchY = trainY.values[selection]

    trainBatchX = np.concatenate([trainBatchX, synthetic_fakes])
    trainBatchY = np.concatenate([trainBatchY, np.ones(500)])

    discriminator.trainable=True
    discriminator.train_on_batch(trainBatchX, trainBatchY)
    discriminator.trainable=False

    noise=np.random.normal(0,1, [500, 100])
    y_gen = np.ones(500)
    gan.train_on_batch(noise, y_gen)

    if e % 1000 == 0:
        print ("Epoch "+str(e/1000))
        
        discriminator.save("./"+model_name+"/model_e"+str(e/1000)+".h5")
        
        predictY = discriminator.predict(testX)
        fpr, tpr, thresholds_roc = roc_curve(testY, predictY)
        roc = pd.DataFrame({'fpr':fpr,'tpr':tpr,'threshold':thresholds_roc})
        roc.to_csv("./"+model_name+"/model_e"+str(e/1000)+"_roc.csv")
        auc_roc_score=roc_auc_score(testY, predictY)

        precision, recall, thresholds_prc = precision_recall_curve(testY, predictY)
        prc = pd.DataFrame({'precision':precision,'recall':recall,'threshold':np.append(thresholds_prc, np.NaN)})
        prc.to_csv("./"+model_name+"/model_e"+str(e/1000)+"_prc.csv")
        auc_prc_score= auc(recall, precision)

        # Pass the row elements as key value pairs to append() function 
        combined = combined.append({"iterations":e/1000, "auroc":auc_roc_score, "auprc": auc_prc_score}, ignore_index=True)


In [ ]:
combined.to_csv("./"+model_name+"/model_score.csv")